### Regression notebook for Wadhwani AI competition

In [1]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from pycocotools import mask as mutils
from pycocotools.coco import COCO
from mmcv import Config
from mmdet.apis import set_random_seed

import random, os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from fastai.vision.all import *
import cv2
import matplotlib.patches as patches

from shapely.wkt import loads
from shapely.ops import clip_by_rect
from shapely.geometry import box
from tqdm import tqdm

/home/rajneesh/miniconda3/envs/pytorch/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
warnings.filterwarnings(action='ignore')

In [15]:
class CFG:
    seed = 46
    debug = False
    n_splits = 5
    
random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)
plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

In [4]:
DIR = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
IMG_PATH = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images'
submit = pd.read_csv(os.path.join(DIR,'SampleSubmission.csv'))
train = pd.read_csv(os.path.join(DIR,'Train.csv'))
test_df = pd.read_csv(os.path.join(DIR,'Test.csv'))
bbox_df = pd.read_csv(os.path.join(DIR,'images_bboxes.csv'))
bbox_df = bbox_df.groupby('image_id').agg(lambda x: list(x)).reset_index()

if CFG.debug:
    bbox_df = bbox_df.iloc[:1000,:]
    test_df = test_df.iloc[:10,:]
    #     train = train.set_index('image_id_worm').loc[bbox_df['image_id']].reset_index()
    
VERSION = "NB_EXP_DETECTION_V0_001"
MODEL_FOLDER = Path(f"///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/{VERSION}/")
os.makedirs(MODEL_FOLDER,exist_ok=True)
KERNEL_TYPE = f"DETECTION_BASELINE"

print(MODEL_FOLDER)
print(KERNEL_TYPE)

/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/NB_EXP_DETECTION_V0_001
DETECTION_BASELINE


In [5]:
def getBBOX_new(df,ix):
    img = df.loc[ix,'image_id']
    geom = df.loc[ix,'geometry']
    label = df.loc[ix,'worm_type']
    img = cv2.imread(f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/{img}')
    H,W,_ = img.shape
    labels = []
    
    bounds_out = []
    for i,g in enumerate(geom):
        bounding_box = loads(str(g))
        image_frame = box(0, 0, W, H)
        valid_box = clip_by_rect(bounding_box, *image_frame.bounds)
        bounds = valid_box.bounds
        
        if len(bounds)!= 4:
            continue
        bounds_out.append(bounds)
        labels.append(label[i])
        
    return bounds_out, H,W,labels

In [6]:
def getBBOX_test(df,ix):
    img = df.loc[ix,'image_id_worm']
    img = cv2.imread(f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/{img}')
    H,W,_ = img.shape        
    return H,W

In [7]:
bbox_copy = bbox_df.copy()
bbox_copy = bbox_copy[~pd.isnull(bbox_copy['geometry'].apply(lambda x: x[0]))].reset_index(drop=True)
bbox_copy = bbox_copy[~pd.isnull(bbox_copy['worm_type'].apply(lambda x: x[0]))].reset_index(drop=True)
print(bbox_copy.shape)

new_bbox = []
Ht = []
Wd = []
ids = []
labels = []

for index in tqdm(np.arange(len(bbox_copy)),total=len(bbox_copy)):
    try:
        id = bbox_copy.loc[index,'image_id']
        bounds,H,W,label = getBBOX_new(bbox_copy,index)
    except:
        continue    

    ids.append(id)
    new_bbox.append(bounds)
    Ht.append(int(H))
    Wd.append(int(W))
    labels.append(label)

(6479, 3)


100%|███████████████████████████████████████████████████████████████████████████████| 6479/6479 [19:14<00:00,  5.61it/s]


In [8]:
Ht_test = []
Wd_test = []
ids_test = []
labels_test = []
new_bbox_test = []
for index in tqdm(np.arange(len(test_df)),total=len(test_df)):
    try:
        id = test_df.loc[index,'image_id_worm']
        H,W = getBBOX_test(test_df,index)
    except:
        continue
        
    ids_test.append(id)
    new_bbox_test.append([])
    Ht_test.append(int(H))
    Wd_test.append(int(W))
    labels_test.append([])

100%|███████████████████████████████████████████████████████████████████████████████| 2803/2803 [06:04<00:00,  7.69it/s]


In [9]:
new_df = pd.concat([pd.Series(ids),pd.Series(new_bbox),pd.Series(Ht),pd.Series(Wd),pd.Series(labels)],1)
new_df.rename(columns={0:'image_id',1:'bbox',2:'Height',3:'Width',4:'label'},inplace=True)
new_df['N'] = new_df['bbox'].apply(lambda x: len(x))
new_df.head(1)

new_df_test = pd.concat([pd.Series(ids_test),pd.Series(new_bbox_test),pd.Series(Ht_test),pd.Series(Wd_test),pd.Series(labels_test)],1)
new_df_test.rename(columns={0:'image_id',1:'bbox',2:'Height',3:'Width',4:'label'},inplace=True)
new_df_test['N'] = new_df_test['bbox'].apply(lambda x: len(x))
new_df_test.head(1)

,image_id,bbox,Height,Width,label,N
0,id_00332970f80fa9a47a39516d.jpg,[],3120,4160,[],0


In [10]:
for image in new_df['image_id']:
    np.savetxt(f'{DIR}/ann_files/{image}.txt',new_df[new_df['image_id']==image].values, fmt=['%s','%s','%d','%d','%s','%d'],delimiter='\n')
    
for image in new_df_test['image_id']:
    np.savetxt(f'{DIR}/ann_files/{image}.txt',new_df_test[new_df_test['image_id']==image].values, fmt=['%s','%s','%d','%d','%s','%d'],delimiter='\n')

In [11]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module(force=True)
class WadhwaniDetectionDataset(CustomDataset):

    CLASSES = ('abw', 'pbw')

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        # convert annotations to middle format
        for image_id in image_list:
            filename = f'{DIR}/images/{image_id}'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}', width=width, height=height)
    
            # load annotations
            l = mmcv.list_from_file(f'{DIR}/ann_files/{image_id}.txt')            
            bbox_names = eval([line.strip().split('\n') for line in l][4][0])
            bboxes = [line.strip().split('\n') for line in l][1][0]
            bboxes = eval(bboxes)
            bboxes = [[float(k) for k in x] for x in bboxes]
            
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            # filter 'DontCare'
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_labels.append(cat2label[bbox_name])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos

In [12]:
import copy
import os.path as osp

import mmcv
import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module(force=True)
class WadhwaniTestDetectionDataset(CustomDataset):

    CLASSES = ('abw', 'pbw')

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        # convert annotations to middle format
        for image_id in image_list:
            filename = f'{DIR}/images/{image_id}'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}', width=width, height=height)
            data_infos.append(data_info)

        return data_infos

### Get kfolds

In [13]:
def make_train_dataset():
    train_new_pbw = pd.DataFrame()
    train_new_pbw['image_id_worm']= train['image_id_worm'].unique()
    train_new_pbw = pd.merge(train_new_pbw,train[train['worm_type']=='pbw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_pbw['worm_type'] = 'pbw'
    train_new_pbw.fillna(0,inplace=True)
    
    train_new_abw = pd.DataFrame()
    train_new_abw['image_id_worm']= train['image_id_worm'].unique()
    train_new_abw = pd.merge(train_new_abw,train[train['worm_type']=='abw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_abw['worm_type'] = 'abw'
    train_new_abw.fillna(0,inplace=True)
    
    train_out = pd.concat([train_new_pbw,train_new_abw],0).reset_index(drop=True)
    
    assert len(train_out) == train['image_id_worm'].nunique()*2
    train_out = pd.pivot(train_out,'image_id_worm','worm_type','number_of_worms').reset_index()
    train_out[['abw','pbw']] = train_out[['abw','pbw']].astype(int)
    
    train_out['abw_cls'] = (train_out['abw']>0).astype(int)
    train_out['pbw_cls'] = (train_out['pbw']>0).astype(int)
    
    return train_out

train_new = make_train_dataset()

In [16]:
mskf = MultilabelStratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=42)
fold_ids = []
train_new['fold'] = 0

for train_index, test_index in mskf.split(train_new, train_new[['abw','pbw']]):
    fold_ids.append(test_index)
    
for fld in range(CFG.n_splits):
    valIx = fold_ids[fld]
    train_new.loc[valIx,'fold']=fld 

In [17]:
new_df = pd.merge(new_df,train_new[['image_id_worm','fold']],left_on='image_id',right_on='image_id_worm',how='left').drop('image_id_worm',1)

In [18]:
bbox_train = new_df[new_df['fold']!= 0].reset_index(drop=True)
bbox_val = new_df[new_df['fold']== 0].reset_index(drop=True)

In [19]:
np.savetxt(f'{DIR}/image_list_files/image_list_train.txt', bbox_train.image_id.values, fmt='%s')
np.savetxt(f'{DIR}/image_list_files/image_list_val.txt', bbox_val.image_id.values, fmt='%s')
np.savetxt(f'{DIR}/image_list_files/image_list_test.txt', new_df_test.image_id.values, fmt='%s')

In [20]:
from mmcv import Config
cfg = Config.fromfile('///home/rajneesh/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

In [21]:
# Modify dataset type and path

cfg.dataset_type = 'WadhwaniDetectionDataset'
cfg.data_root = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'

cfg.data.test.type = 'WadhwaniDetectionDataset'
cfg.data.test.data_root = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
cfg.data.test.ann_file = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/image_list_files/image_list_test.txt'
cfg.data.test.img_prefix = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/'

cfg.data.train.type = 'WadhwaniDetectionDataset'
cfg.data.train.data_root = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
cfg.data.train.ann_file = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/image_list_files/image_list_train.txt'
cfg.data.train.img_prefix = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/'

cfg.data.val.type = 'WadhwaniDetectionDataset'
cfg.data.val.data_root = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
cfg.data.val.ann_file = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/image_list_files/image_list_val.txt'
cfg.data.val.img_prefix = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/'

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 2
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
# cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# Set up working dir to save files and logs.
cfg.work_dir = f'{MODEL_FOLDER}'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

cfg.runner = {'type': 'EpochBasedRunner', 'max_epochs': 15}
cfg.data.samples_per_gpu = 8 
cfg.workers_per_gpu = 8
cfg.lr_config.step = [11,14]

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 5
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'

In [22]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# Build dataset
datasets = [build_dataset(cfg.data.train)]

Invalid SOS parameters for sequential JPEG


In [23]:
input = datasets.__getitem__(0)
input.data_infos[0]

{'filename': 'id_0005ef295aafe6acc63587db.jpg',
 'width': 3120,
 'height': 4160,
 'ann': {'bboxes': array([[1587.45, 2631.2 , 1685.22, 2979.61],
         [1296.7 , 1850.31, 1451.04, 2063.9 ],
         [1547.51, 2321.35, 1653.55, 2508.64],
         [1363.07, 1169.51, 1579.03, 1288.13],
         [1639.78, 2292.43, 1770.61, 2516.9 ],
         [ 709.83, 1055.61,  900.54, 1181.43],
         [ 654.54, 2289.89,  902.58, 2405.64],
         [ 701.36,  940.51,  910.12, 1089.47]], dtype=float32),
  'labels': array([1, 1, 1, 1, 1, 1, 1, 1]),
  'bboxes_ignore': array([], shape=(0, 4), dtype=float32),
  'labels_ignore': array([], dtype=int64)}}

In [24]:
input.get_ann_info(0)

{'bboxes': array([[1587.45, 2631.2 , 1685.22, 2979.61],
        [1296.7 , 1850.31, 1451.04, 2063.9 ],
        [1547.51, 2321.35, 1653.55, 2508.64],
        [1363.07, 1169.51, 1579.03, 1288.13],
        [1639.78, 2292.43, 1770.61, 2516.9 ],
        [ 709.83, 1055.61,  900.54, 1181.43],
        [ 654.54, 2289.89,  902.58, 2405.64],
        [ 701.36,  940.51,  910.12, 1089.47]], dtype=float32),
 'labels': array([1, 1, 1, 1, 1, 1, 1, 1]),
 'bboxes_ignore': array([], shape=(0, 4), dtype=float32),
 'labels_ignore': array([], dtype=int64)}

In [25]:
input.prepare_train_img(0)

{'img_metas': DataContainer({'filename': '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images/id_0005ef295aafe6acc63587db.jpg', 'ori_filename': 'id_0005ef295aafe6acc63587db.jpg', 'ori_shape': (4160, 3120, 3), 'img_shape': (1067, 800, 3), 'pad_shape': (1088, 800, 3), 'scale_factor': array([0.25641027, 0.25649038, 0.25641027, 0.25649038], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}),
 'img': DataContainer(tensor([[[0.5536, 0.5878, 0.5878,  ..., 0.9646, 1.1358, 1.0159],
          [0.5364, 0.5536, 0.6049,  ..., 0.9474, 1.0502, 1.0844],
          [0.5536, 0.5878, 0.6221,  ..., 0.7933, 1.0159, 0.9817],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0

In [26]:
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

In [27]:
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-11-11 13:14:43,783 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-11 13:16:37,444 - mmdet - INFO - Start running, host: rajneesh@Rajneesh, work_dir: /mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/NB_EXP_DETECTION_V0_001
2022-11-11 13:16:37,444 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook        

2022-11-11 13:18:39,795 - mmdet - INFO - Epoch [1][250/648]	lr: 1.000e-03, eta: 1:17:10, time: 0.455, data_time: 0.026, memory: 15992, loss_rpn_cls: 0.2957, loss_rpn_bbox: 0.1021, loss_cls: 0.2378, acc: 93.2300, loss_bbox: 0.1627, loss: 0.7982
2022-11-11 13:18:44,155 - mmdet - INFO - Epoch [1][260/648]	lr: 1.000e-03, eta: 1:16:47, time: 0.440, data_time: 0.028, memory: 15992, loss_rpn_cls: 0.2697, loss_rpn_bbox: 0.0925, loss_cls: 0.2117, acc: 94.1260, loss_bbox: 0.1422, loss: 0.7161
2022-11-11 13:18:48,920 - mmdet - INFO - Epoch [1][270/648]	lr: 1.000e-03, eta: 1:16:38, time: 0.475, data_time: 0.048, memory: 15992, loss_rpn_cls: 0.2835, loss_rpn_bbox: 0.1047, loss_cls: 0.2557, acc: 92.3486, loss_bbox: 0.2058, loss: 0.8497
2022-11-11 13:18:53,537 - mmdet - INFO - Epoch [1][280/648]	lr: 1.000e-03, eta: 1:16:24, time: 0.461, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.2655, loss_rpn_bbox: 0.0872, loss_cls: 0.2317, acc: 93.4155, loss_bbox: 0.1842, loss: 0.7686
2022-11-11 13:18:58,188 

2022-11-11 13:21:17,445 - mmdet - INFO - Epoch [1][590/648]	lr: 1.000e-03, eta: 1:12:11, time: 0.457, data_time: 0.024, memory: 15992, loss_rpn_cls: 0.1734, loss_rpn_bbox: 0.0944, loss_cls: 0.2671, acc: 90.5884, loss_bbox: 0.2761, loss: 0.8110
2022-11-11 13:21:22,176 - mmdet - INFO - Epoch [1][600/648]	lr: 1.000e-03, eta: 1:12:06, time: 0.471, data_time: 0.023, memory: 15992, loss_rpn_cls: 0.2349, loss_rpn_bbox: 0.1049, loss_cls: 0.2678, acc: 90.9155, loss_bbox: 0.2499, loss: 0.8576
2022-11-11 13:21:26,862 - mmdet - INFO - Epoch [1][610/648]	lr: 1.000e-03, eta: 1:12:00, time: 0.469, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.2007, loss_rpn_bbox: 0.0916, loss_cls: 0.2397, acc: 92.1533, loss_bbox: 0.2171, loss: 0.7491
2022-11-11 13:21:31,590 - mmdet - INFO - Epoch [1][620/648]	lr: 1.000e-03, eta: 1:11:55, time: 0.474, data_time: 0.026, memory: 15992, loss_rpn_cls: 0.1588, loss_rpn_bbox: 0.0938, loss_cls: 0.2483, acc: 91.3037, loss_bbox: 0.2536, loss: 0.7544
2022-11-11 13:21:36,159 

2022-11-11 13:24:02,656 - mmdet - INFO - Epoch [2][290/648]	lr: 1.000e-03, eta: 1:08:43, time: 0.453, data_time: 0.026, memory: 15992, loss_rpn_cls: 0.1615, loss_rpn_bbox: 0.0985, loss_cls: 0.2746, acc: 89.9658, loss_bbox: 0.3223, loss: 0.8569
2022-11-11 13:24:07,004 - mmdet - INFO - Epoch [2][300/648]	lr: 1.000e-03, eta: 1:08:35, time: 0.434, data_time: 0.024, memory: 15992, loss_rpn_cls: 0.1329, loss_rpn_bbox: 0.0753, loss_cls: 0.2729, acc: 90.5127, loss_bbox: 0.3306, loss: 0.8116
2022-11-11 13:24:11,700 - mmdet - INFO - Epoch [2][310/648]	lr: 1.000e-03, eta: 1:08:31, time: 0.470, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.1593, loss_rpn_bbox: 0.1054, loss_cls: 0.2804, acc: 90.0415, loss_bbox: 0.3256, loss: 0.8706
2022-11-11 13:24:16,297 - mmdet - INFO - Epoch [2][320/648]	lr: 1.000e-03, eta: 1:08:25, time: 0.460, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.1317, loss_rpn_bbox: 0.0792, loss_cls: 0.2656, acc: 90.9326, loss_bbox: 0.2935, loss: 0.7700
Invalid SOS parameters f

2022-11-11 13:26:40,309 - mmdet - INFO - Epoch [2][630/648]	lr: 1.000e-03, eta: 1:05:50, time: 0.468, data_time: 0.027, memory: 15992, loss_rpn_cls: 0.1332, loss_rpn_bbox: 0.1056, loss_cls: 0.2959, acc: 89.3604, loss_bbox: 0.3371, loss: 0.8719
2022-11-11 13:26:44,825 - mmdet - INFO - Epoch [2][640/648]	lr: 1.000e-03, eta: 1:05:45, time: 0.452, data_time: 0.026, memory: 15992, loss_rpn_cls: 0.1400, loss_rpn_bbox: 0.0946, loss_cls: 0.2980, acc: 89.7534, loss_bbox: 0.3298, loss: 0.8624
2022-11-11 13:26:48,354 - mmdet - INFO - Saving checkpoint at 2 epochs
2022-11-11 13:26:57,222 - mmdet - INFO - Epoch [3][10/648]	lr: 1.000e-03, eta: 1:05:31, time: 0.754, data_time: 0.305, memory: 15992, loss_rpn_cls: 0.1660, loss_rpn_bbox: 0.1094, loss_cls: 0.3211, acc: 89.0234, loss_bbox: 0.3361, loss: 0.9326
2022-11-11 13:27:01,836 - mmdet - INFO - Epoch [3][20/648]	lr: 1.000e-03, eta: 1:05:25, time: 0.459, data_time: 0.023, memory: 15992, loss_rpn_cls: 0.1357, loss_rpn_bbox: 0.0869, loss_cls: 0.2852, a

...

2022-11-11 14:28:12,302 - mmdet - INFO - Epoch [14][440/648]	lr: 1.000e-04, eta: 0:06:36, time: 0.456, data_time: 0.027, memory: 15992, loss_rpn_cls: 0.0761, loss_rpn_bbox: 0.0924, loss_cls: 0.2910, acc: 87.9834, loss_bbox: 0.4002, loss: 0.8597
2022-11-11 14:28:16,945 - mmdet - INFO - Epoch [14][450/648]	lr: 1.000e-04, eta: 0:06:31, time: 0.463, data_time: 0.023, memory: 15992, loss_rpn_cls: 0.0849, loss_rpn_bbox: 0.0673, loss_cls: 0.2803, acc: 89.4067, loss_bbox: 0.3597, loss: 0.7922
2022-11-11 14:28:21,537 - mmdet - INFO - Epoch [14][460/648]	lr: 1.000e-04, eta: 0:06:27, time: 0.458, data_time: 0.026, memory: 15992, loss_rpn_cls: 0.0882, loss_rpn_bbox: 0.0889, loss_cls: 0.2844, acc: 88.6182, loss_bbox: 0.3936, loss: 0.8551
2022-11-11 14:28:26,282 - mmdet - INFO - Epoch [14][470/648]	lr: 1.000e-04, eta: 0:06:22, time: 0.474, data_time: 0.027, memory: 15992, loss_rpn_cls: 0.0874, loss_rpn_bbox: 0.0905, loss_cls: 0.3016, acc: 87.7124, loss_bbox: 0.4370, loss: 0.9165
2022-11-11 14:28:30,

2022-11-11 14:30:57,906 - mmdet - INFO - Epoch [15][140/648]	lr: 1.000e-05, eta: 0:03:55, time: 0.474, data_time: 0.027, memory: 15992, loss_rpn_cls: 0.0639, loss_rpn_bbox: 0.0639, loss_cls: 0.2532, acc: 89.9951, loss_bbox: 0.3422, loss: 0.7231
2022-11-11 14:31:02,546 - mmdet - INFO - Epoch [15][150/648]	lr: 1.000e-05, eta: 0:03:50, time: 0.464, data_time: 0.024, memory: 15992, loss_rpn_cls: 0.1057, loss_rpn_bbox: 0.0884, loss_cls: 0.3080, acc: 87.4536, loss_bbox: 0.3931, loss: 0.8952
2022-11-11 14:31:07,223 - mmdet - INFO - Epoch [15][160/648]	lr: 1.000e-05, eta: 0:03:45, time: 0.466, data_time: 0.024, memory: 15992, loss_rpn_cls: 0.0845, loss_rpn_bbox: 0.1045, loss_cls: 0.3077, acc: 87.4634, loss_bbox: 0.3868, loss: 0.8835
2022-11-11 14:31:11,828 - mmdet - INFO - Epoch [15][170/648]	lr: 1.000e-05, eta: 0:03:41, time: 0.461, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.0779, loss_rpn_bbox: 0.0755, loss_cls: 0.2568, acc: 89.8169, loss_bbox: 0.3457, loss: 0.7559
2022-11-11 14:31:16,

2022-11-11 14:33:35,244 - mmdet - INFO - Epoch [15][480/648]	lr: 1.000e-05, eta: 0:01:17, time: 0.457, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.0947, loss_rpn_bbox: 0.1038, loss_cls: 0.2992, acc: 87.7734, loss_bbox: 0.4247, loss: 0.9224
2022-11-11 14:33:39,912 - mmdet - INFO - Epoch [15][490/648]	lr: 1.000e-05, eta: 0:01:13, time: 0.466, data_time: 0.025, memory: 15992, loss_rpn_cls: 0.1085, loss_rpn_bbox: 0.1117, loss_cls: 0.3341, acc: 86.0596, loss_bbox: 0.4328, loss: 0.9870
2022-11-11 14:33:44,600 - mmdet - INFO - Epoch [15][500/648]	lr: 1.000e-05, eta: 0:01:08, time: 0.468, data_time: 0.028, memory: 15992, loss_rpn_cls: 0.0780, loss_rpn_bbox: 0.0941, loss_cls: 0.2973, acc: 88.1226, loss_bbox: 0.3857, loss: 0.8551
2022-11-11 14:33:49,237 - mmdet - INFO - Epoch [15][510/648]	lr: 1.000e-05, eta: 0:01:03, time: 0.465, data_time: 0.027, memory: 15992, loss_rpn_cls: 0.0763, loss_rpn_bbox: 0.1107, loss_cls: 0.3073, acc: 87.1680, loss_bbox: 0.3916, loss: 0.8859
2022-11-11 14:33:53,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1301/1301, 19.9 task/s, elapsed: 65s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-11-11 14:35:59,742 - mmdet - INFO - 
+-------+-------+-------+--------+-------+
| class | gts   | dets  | recall | ap    |
+-------+-------+-------+--------+-------+
| abw   | 1787  | 8976  | 0.919  | 0.779 |
| pbw   | 27404 | 78260 | 0.381  | 0.212 |
+-------+-------+-------+--------+-------+
| mAP   |       |       |        | 0.495 |
+-------+-------+-------+--------+-------+
2022-11-11 14:35:59,764 - mmdet - INFO - Epoch(val) [15][1301]	AP50: 0.4950, mAP: 0.4954


In [ ]:
# !ls '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/NB_EXP_DETECTION_V0_001/latest.pth'

In [28]:
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

checkpoint = load_checkpoint(model, '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/NB_EXP_DETECTION_V0_001/latest.pth'
                             , map_location='cpu')

load checkpoint from local path: ///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/models/NB_EXP_DETECTION_V0_001/latest.pth


In [29]:
from mmdet.apis.test import single_gpu_test
from mmdet.datasets import build_dataloader
# build the dataloader
dataset_test = build_dataset(cfg.data.test)

Invalid SOS parameters for sequential JPEG


In [30]:
data_loader = build_dataloader(
    dataset_test,
    samples_per_gpu=1,
    workers_per_gpu=1,
    dist=False,
    shuffle=False)

In [31]:
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader, show=False)

[>>>>>>>>>>>>>>>>>         ] 1856/2803, 10.2 task/s, elapsed: 181s, ETA:    92s

Invalid SOS parameters for sequential JPEG


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 2803/2803, 10.3 task/s, elapsed: 271s, ETA:     0s

In [34]:
len(outputs),len(test_df)

(2803, 2803)

In [48]:
# outputs

In [45]:
ABW = []
PBW = []
for imgs in tqdm(range(len(outputs))):
    l = outputs[imgs]
    N_ABW = len(l[0])
    N_PBW = len(l[1])
    ABW.append(N_ABW)
    PBW.append(N_PBW)    

100%|██████████████████████████████████████████████████████████████████████████| 2803/2803 [00:00<00:00, 1994001.72it/s]


In [50]:
test_df['abw'] = ABW
test_df['pbw'] = PBW
test_df.head()

,image_id_worm,abw,pbw
0,id_00332970f80fa9a47a39516d.jpg,9,7
1,id_0035981bc3ae42eb5b57a317.jpg,0,80
2,id_005102f664b820f778291dee.jpg,20,0
3,id_0066456f5fb2cd858c69ab39.jpg,17,8
4,id_007159c1fa015ba6f394deeb.jpg,1,99


In [57]:
test_df_melt = test_df.melt('image_id_worm',['abw','pbw'],value_name='number_of_worms')
test_df_melt['image_id_worm'] = test_df_melt['image_id_worm'].apply(lambda x: x.split('.')[0])
test_df_melt['image_id_worm'] = test_df_melt['image_id_worm']+"_"+test_df_melt['variable']
test_df_melt.drop(['variable'],1,inplace=True)
test_df_melt.tail()

,image_id_worm,number_of_worms
5601,id_ffad8f3773a4222f8fe5ba1a_pbw,38
5602,id_ffb65e6de900c49d8f2ef95a_pbw,96
5603,id_ffbcb27fa549278f47505515_pbw,0
5604,id_ffc0e41e10b0c964d4a02811_pbw,22
5605,id_fff8c253115aacded09ad7ed_pbw,43


In [59]:
submit = pd.merge(submit.drop('number_of_worms',1),test_df_melt,on='image_id_worm',how='left')
submit.head()

,image_id_worm,number_of_worms
0,id_00332970f80fa9a47a39516d_abw,9
1,id_00332970f80fa9a47a39516d_pbw,7
2,id_0035981bc3ae42eb5b57a317_abw,0
3,id_0035981bc3ae42eb5b57a317_pbw,80
4,id_005102f664b820f778291dee_abw,20


In [60]:
submit.to_csv(f'{MODEL_FOLDER}/detection_preds.csv',index=False)

### Fin 